In [94]:
!pip install lda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [95]:
import nltk
import re
import collections
import itertools
import lda
import requests
import csv
import time
import math
import operator
import numpy as np
import pandas as pd
from collections import defaultdict
from pandas import read_table
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [96]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score    # 정확도 계산

In [97]:
df = pd.read_csv('/content/drive/MyDrive/disney.csv')
df

,Unnamed: 0,rating,review,date,userName,title
0,0,5,컨텐츠야 디즈니니까 당연히 넘쳐나는데 검색 기능이 상대적으로 부실합니다. 예를 들면...,2021-11-11 20:33:32,악틱,좋습니당. 몇 가지 건의 사항
1,1,5,디즈니 만화와 영화를 사랑하는 사람으로써 몇 가지 개선해주셨으면 하는 바가 있어 남...,2021-11-15 12:15:12,이제가을,디즈니를 아끼는 마음으로 남겨요~
2,2,5,1년권 끊고 한 달만에 후회했는데 문나이트로 뽕 뽑음. 문나이트만 한 다섯번째 돌...,2022-06-06 05:58:35,양털끄트머리,하나만 걸려도 충분하다
3,3,3,전반 적으로 만족 합니다만 넷플이나 기타 스트리밍 앱에서 되는 것들이 안되는게 있어...,2021-11-14 06:14:34,filco518,기대하던 디플 앱 드디어 사용 해봤네요
4,4,5,성인인증 시 프로필 PIN 넘버 입력 키보드 자판이 안뜨고 구글 크롬캐스트4 사용 ...,2021-11-11 23:59:48,무무니,컨텐츠 많아서 좋긴한데
...,...,...,...,...,...,...
4535,3457,5,짱,2022-03-12 20:00:39,김지혜,NaN
4536,3458,5,👍,2022-01-14 16:35:11,정윤재,NaN
4537,3459,1,Good 😉😉😉😉😉😉😉😉😉😉😉,2021-03-23 13:07:45,Jacob Won,NaN
4538,3460,1,1000000000000000000000000000000000000000000000...,2022-01-24 21:45:05,황은비,NaN


In [98]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [99]:
df[df['review'].str.contains('뭉탱이') == True]['review']

1050    뭉탱이로있다가유리게슝아니그냥뭉탱이로있다가유리게슝아니그냥뭉탱이로있다가유리게슝아니그냥뭉...
Name: review, dtype: object

In [100]:
df[df['review'].str.contains('레이디') == True]['review']

986    왜 레이디버그 없어요?  왜 레이디버그  왜 레이디버그 없어요? 왜 레이디버그 없어...
Name: review, dtype: object

In [101]:
'''
data_list = df['review']
data_list
'''

"\ndata_list = df['review']\ndata_list\n"

In [102]:

data_list = df['review'].tolist()
data_list


['컨텐츠야 디즈니니까 당연히 넘쳐나는데 검색 기능이 상대적으로 부실합니다. 예를 들면 다른 ott들은 A를 검색했을때 그 영화가 없더라도 비슷한 작품을 띄워주잖아요. 디즈니는 그런 것 없이 정직하게 없다고만 떠서 아쉬웠습니다. \n그 외로는 배속 기능이 없다는 점과, 자막 크기가 폰에서는 작고 패드에선 커서 소소하게 불편을 느낍니다. 자막 크기와 스타일을 조절할 수 있게 한다면 더할 나위 없이 좋겠네요. 일반적인 영화 자막과는 달리 검은 바탕을 깔아놓아서 유튜브 영상을 보는 것 같다는 느낌을 받았어요. \n한국 출시 첫 날이라 그런지 어플이 전반적으로 살짝 어설프다는 생각이 듭니다. 미국 디즈니플러스 어플은 써보지 않아서 그쪽도 한국 어플과 같은지 궁금하네요. 아무튼 일년 간 잘 써볼게요',
 '디즈니 만화와 영화를 사랑하는 사람으로써 몇 가지 개선해주셨으면 하는 바가 있어 남깁니다. 스마트tv에서 볼때 자막크기가 스크린을 방해하는 느낌이 듭니다. 100프로는 너무 작고, 200프로는 너무 커서 그 중간사이 150% 비율의 폰트 사이즈가 있었으면 좋겠어요~그리고 또하나는 화면에서 터치로 음량이나 화면밝기를 조절할 수 없다는게 아쉽네요 이것들만 개선될 수 있다면 나머지는 대만족입니다:) 오래오래 디즈니플러스 구독할 수 있도록 꼭 개선될 수 있길 바랄게요😊',
 '1년권 끊고 한 달만에 후회했는데 문나이트로 뽕 뽑음.  문나이트만 한 다섯번째 돌려보는 중이다. 진짜 취향에 맞는 작품 하나만 걸려도 ott는 돈이 안 아까움.',
 '전반 적으로 만족 합니다만 넷플이나 기타 스트리밍 앱에서 되는 것들이 안되는게 있어서 별 1개 뺐네요 \n특히 아쉬운게 한번이라도 재생 하면 지금 시청중 목록에 이어보기 할수 있게 쌓여 가는데 보기 싫은거 지울수가 없는 부분이 아쉬움\n무한정 계속 쌓이는것 같고... 넷플처럼 시청기록 자체를 지우는것 까진 없어도 \n시청중 목록 지울수 있게 해주는 기능 추가가 시급해 보여요\n미국이나 다른 나라에서 벌써 2년 째 운영 중인데 왜 이런 

In [103]:
pos_list = df[df['rating'] >= 4]
neg_list = df[df['rating'] <= 2]

In [104]:
pos_corpus = np.array(pos_list['review'])
neg_corpus = np.array(neg_list['review'])

In [105]:
total_data = np.concatenate((pos_corpus,neg_corpus))

In [106]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 3s (95.8 kB/s)
Reading package li

In [107]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [108]:
from konlpy.tag import Okt
import nltk

In [109]:
okt=Okt()

In [110]:
pos_reviews = ''
   
for i in pos_list['review'] : 
  pos_reviews = pos_reviews + i + '\n'

In [111]:
tokens = okt.morphs(pos_reviews)

In [112]:
result = nltk.Text(tokens)

In [113]:
review_pos = okt.pos(pos_reviews)

In [114]:
pos_df = pd.DataFrame(review_pos)
pos_df.columns = ['token', 'pos']
pos_df.head()

,token,pos
0,컨텐츠,Noun
1,야,Josa
2,디즈니,Noun
3,니까,Josa
4,당연히,Adjective


In [115]:
noun = pos_df[pos_df['pos']== 'Noun']['token']
josa = pos_df[pos_df['pos']== 'Josa']['token']
verbprefix = pos_df[pos_df['pos']== 'Verbprefix']['token']
verb = pos_df[pos_df['pos']== 'Verb']['token']
punctuation = pos_df[pos_df['pos']== 'Punctuation']['token']
adjective = pos_df[pos_df['pos']== 'Adjective']['token']
suffix = pos_df[pos_df['pos']== 'Suffix']['token']
number = pos_df[pos_df['pos']== 'Number']['token']
adverb = pos_df[pos_df['pos']== 'Adverb']['token']
conjunction = pos_df[pos_df['pos']== 'Conjunction']['token']
alpha = pos_df[pos_df['pos']== 'Alpha']['token']
foreign = pos_df[pos_df['pos']== 'Foreign']['token']
determiner = pos_df[pos_df['pos']== 'Determiner']['token']
modifier = pos_df[pos_df['pos']== 'Modifier']['token']
koreanparticle = pos_df[pos_df['pos']== 'KoreanParticle']['token']
eomi = pos_df[pos_df['pos']== 'Eomi']['token']
exclamation = pos_df[pos_df['pos']== 'Exclamation']['token']
url = pos_df[pos_df['pos']== 'URL']['token']
hashtag = pos_df[pos_df['pos']== 'Hashtag']['token']
email = pos_df[pos_df['pos']== 'Email']['token']
preeomi = pos_df[pos_df['pos']== 'PreEomi']['token']
screenname = pos_df[pos_df['pos']== 'ScreenName']['token']

In [116]:
noun_list = noun.to_list()

In [117]:
# 불용어 처리

In [118]:
total_data.shape

(3959,)

In [119]:
c_vetorizer = CountVectorizer(analyzer='word')
count = c_vetorizer.fit_transform(noun_list)

In [120]:
model = lda.LDA(n_topics = 4, n_iter = 1000)
model.fit(count)

INFO:lda:n_documents: 10657
INFO:lda:vocab_size: 1745
INFO:lda:n_words: 8204
INFO:lda:n_topics: 4
INFO:lda:n_iter: 1000
INFO:lda:<0> log likelihood: -70698
INFO:lda:<10> log likelihood: -60718
INFO:lda:<20> log likelihood: -59948
INFO:lda:<30> log likelihood: -59457
INFO:lda:<40> log likelihood: -58913
INFO:lda:<50> log likelihood: -58837
INFO:lda:<60> log likelihood: -58543
INFO:lda:<70> log likelihood: -58588
INFO:lda:<80> log likelihood: -58424
INFO:lda:<90> log likelihood: -58487
INFO:lda:<100> log likelihood: -58686
INFO:lda:<110> log likelihood: -58517
INFO:lda:<120> log likelihood: -58345
INFO:lda:<130> log likelihood: -58235
INFO:lda:<140> log likelihood: -58231
INFO:lda:<150> log likelihood: -58220
INFO:lda:<160> log likelihood: -58351
INFO:lda:<170> log likelihood: -58109
INFO:lda:<180> log likelihood: -58160
INFO:lda:<190> log likelihood: -58212
INFO:lda:<200> log likelihood: -58363
INFO:lda:<210> log likelihood: -58412
INFO:lda:<220> log likelihood: -58046
INFO:lda:<230> lo

In [121]:
topic_vocab = c_vetorizer.get_feature_names()
topic_word = model.topic_word_
n_top_word = 300
dist = []
for i, topic_dist in enumerate(topic_word):
    dist.append(topic_dist)
    topic_words = np.array(topic_vocab)[np.argsort(topic_dist)][:-n_top_word:-1]
    print('Topic', i+1, topic_words)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic 1 ['디즈니' '배속' '시청' '제발' '최고' '밝기' '어플' '작품' '스타워즈' '부분' '번역' '화질' '바로' '아주'
 '한글' '조금' '서비스' '검색' '모바일' '이용' '크롬캐스트' '버전' '오디오' '리뷰' '동시' '취소' '완전'
 '안보' '혹시' '제일' '아이폰' '감사' '팝업' '미러링' '핸드폰' '성인' '아예' '편의' '방법' '휴대폰' '불편'
 '로그인' '업로드' '음량' '런칭' '디플' '꿀잼' '상륙' '태블릿' '처음' '버퍼링' '자꾸' '평점' '인터넷'
 '다운로드' '자주' '장르' '위로' '사용' '화면' '리안' '중인' '연동' '치면' '마지막' '제대로' '얼른'
 '검은색' '정도' '기준' '수도' '볼땐' '몇개' '나라' '완다' '장점' '작고' '데이터' '개봉' '출시' '적극'
 '난쟁이' '메이' '방영' '공간' '고전' '퀄리티' '여요' '솔져' '필수' '수가' '당황' '애니매이션' '피드백'
 '고화질' '광팬' '보드' '스킵' '유저' '보람' '몰입' '최고다' '조작' '채팅' '현상' '어벤져스' '미드' '방식'
 '감상' '발열' '기본' '차차' '갤럭시' '요청' '로컬라이징' '듭니' '반복' '백설공주' '티빙' '저작권' '관련'
 '대신' '마다' '안함' '구리' '발전' '바탕' '쿠폰' '군사정권' '엄마' '아래' '금방' '나위' '아쉬움' '배터리'
 '정독' '번역가' '지금껏' '접근성' '약간' '키즈' '주시' '제작자' '구독료' '뮤지컬' '기사' '버그' '구간'
 '괴짜가족' '실컷' '미키' '튕겨' '그동안' '어쨌든' '난후' '대응' '또한' '강아지' '로만' '부족함' '동기화'
 '각종' '화이팅' '만원' '화가' '하나로' '의견' '옛날' '레이' '북한' '걱정' '생기' '이름' '드가자' '한국영'
 '인지' '멀티' '설마' '만해' '오히려' '포기' '이예' '와이파이

In [122]:
model.topic_word_

array([[4.78594846e-06, 4.78594846e-06, 4.78594846e-06, ...,
        4.83380794e-04, 4.78594846e-06, 4.78594846e-06],
       [4.90328275e-06, 4.90328275e-06, 4.90328275e-06, ...,
        4.90328275e-06, 4.90328275e-06, 9.85559832e-04],
       [4.83687635e-06, 9.72212145e-04, 3.39065032e-03, ...,
        4.83687635e-06, 4.88524511e-04, 4.83687635e-06],
       [4.33704782e-03, 4.81359359e-06, 4.81359359e-06, ...,
        4.81359359e-06, 4.81359359e-06, 4.81359359e-06]])

In [123]:
stopword=['디즈니','왜','좀','수','볼','것','그냥','슝','더','땡','보고','점','거','진짜','요','다른','안','이','제','저','그','등','내','시','개','전','폰','뭉탱이','바로',
'다시','달','계속','아주','경우','뭐','정도','일단','앞','한번','때문','못','게','도','언제','편','레이디','유리',
'나','중','함','다만','정','이건','꼭','디','분','외','걸','흠','등등','후','로','완전','이제','그거','매우','건','여기','애','또','지금','은','아니','있다가','입니다','있는','같아요','있어서','부탁드립니다','좋겠습니다','있는데','같습니다','같은','있습니다','있게','있었으면','있어','있고',
'어떻게','같네요']

In [124]:
from collections import defaultdict
from pandas import read_table
import numpy as np
import math

class NaiveBayesClassifier:

    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []

    def load_corpus(self, path):
        corpus = read_table(path, sep=',', encoding='utf-8')
        corpus = np.array(corpus)
        return corpus

    def count_words(self, training_set):
        # 학습데이터는 영화리뷰 본문(doc), 평점(point)으로 구성
        counts = defaultdict(lambda : [0, 0])
        for doc, point in training_set:
            # 영화리뷰가 text일 때만 카운트
            if self.isNumber(doc) is False:
                

                # 리뷰를 띄어쓰기 단위로 토크나이징
                words = doc.split()

                data_pos = okt.pos(words)
                words_nnp = [word for word,pos in data_pos if pos in ['Noun','Adjective']]
                words = [w for w in words_nnp if not w in stopword]



                for word in words:
                    counts[word][0 if point > 3.5 else 1] += 1
        return counts

    def isNumber(self, s):
        try:
            float(s)
            return True
        except ValueError:
            return False

    def word_probabilities(self, counts, total_class0, total_class1, k):
        # 단어의 빈도수를 [단어, p(w|긍정), p(w|부정)] 형태로 반환
        return [(w,
                 (class0 + k) / (total_class0 + 2*k),
                 (class1 + k) / (total_class1 + 2*k))
                for w, (class0, class1) in counts.items()]

    def class0_probability(self, word_probs, doc):
        # 별도 토크나이즈 안하고 띄어쓰기로만
        docwords = doc.split()

        # 초기값은 모두 0으로 처리
        log_prob_if_class0 = log_prob_if_class1 = 0.0

        # 모든 단어에 대해 반복
        for word, prob_if_class0, prob_if_class1 in word_probs:
            # 만약 리뷰에 word가 나타나면
            # 해당 단어가 나올 log 확률을 더해 줌
            if word in docwords:
                log_prob_if_class0 += math.log(prob_if_class0)
                log_prob_if_class1 += math.log(prob_if_class1)

            # 만약 리뷰에 word가 나타나지 않는다면
            # 해당 단어가 나오지 않을 log 확률을 더해 줌
            # 나오지 않을 확률은 log(1-나올 확률)로 계산
            else:
                log_prob_if_class0 += math.log(1.0 - prob_if_class0)
                log_prob_if_class1 += math.log(1.0 - prob_if_class1)

        prob_if_class0 = math.exp(log_prob_if_class0)
        prob_if_class1 = math.exp(log_prob_if_class1)
        return prob_if_class0 / (prob_if_class0 + prob_if_class1)

    def train(self, trainfile_path):
        training_set = self.load_corpus(trainfile_path)

        # 범주0(긍정)과 범주1(부정) 문서 수를 세어 줌
        num_class0 = len([1 for _, point in training_set if point > 3.5])
        num_class1 = len(training_set) - num_class0

        # train
        word_counts = self.count_words(training_set)
        self.word_probs = self.word_probabilities(word_counts,
                                                  num_class0,
                                                  num_class1,
                                                  self.k)

    def classify(self, doc):
        return self.class0_probability(self.word_probs, doc)

In [125]:
disney = pd.read_csv('/content/drive/MyDrive/disney.csv')

In [126]:
disney = disney[['review', 'rating']]

In [127]:
disney.to_csv('filtered.csv',index=None,header=None)

In [128]:
model = NaiveBayesClassifier()
model.train(trainfile_path='/content/filtered.csv')

AssertionError: ignored

In [ ]:
model.classify('넷플릭스가 더 좋은듯')

In [ ]:
# 긍정 단어 가중치로 딕셔너리 만들기
dict_p = {}
for i in model.word_probs:
    dict_p[i[0]] = i[1]
# 부정 단어 가중치로 딕셔너리 만들기    
dict_n = {}
for i in model.word_probs:
    dict_n[i[0]] = i[2]
# 각 딕셔너리 값을 내림차순 정렬
sort_dict_p = sorted(dict_p.items(), key=operator.itemgetter(1), reverse=True)
sort_dict_n = sorted(dict_n.items(), key=operator.itemgetter(1), reverse=True)

In [ ]:
# 각 리뷰별 상위 단어 추출
for i in sort_dict_p[0:30]:
    print(i[0])

In [ ]:
for i in sort_dict_n[0:30]:
    print(i[0])